In [13]:
#by bauto: boceto del eda 
# dejo de subtitulo EDA-pro con sugerencias a ejecutar

# 🔎 Análisis Exploratorio de Datos:

* **EDA-Calidad po-rubrica en verde todo gud:**
  1. Tipos de datos
  2. Valores faltantes
  3. Nulos
  4. Duplicados
  5. metadatos/dicc datos
  6. outliers
  7. distribucion 


* **EDA-pro plus, henry contratame quiero vivir de esto:**
  
  1. Cantidad de valores totales en cada columna
  2. Tamaño de los datos
  3. Estadistica descriptiva(cantidad,media,desviacion estandar,minimo,percentil,mediana,percentil 75, màximo)
  4. Varianza
  5. ☁️ Nube de palabras 
  6. Correlaciones entre variables
  7. Distribuciones de las variables numéricas
  8. Distribuciones de las variables categóricas
  9. Análisis de serie de tiempo

Tambien vamos a estar utilizando la funcion personalizada personalizada `data_type_check` invocada desde `data_utils.py` para dejar un vistazo raápido del dataframe y  poder observar:
- Variables categóricas
- Variables numéricas
- Dimensiones del dataframe
- Nulos
- Tipos de datos
- Informacion acerca de los datos faltantes o nulos de cada columna    


####  **Importamos las librerías que vamos a usar**


In [2]:
import gc
import os
import warnings

import pandas as pd
from data_utils import data_type_check, data_type_check_pkl

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import seaborn as sns

#### 📦 **Extraccion** de los datos y primera exploración 


In [3]:
#Yelp
business = pd.read_parquet('../0_Dataset/Yelp/business.parquet')
df_rev_FL = pd.read_parquet('../0_Dataset/Yelp/review_reducido.parquet')
df_checkin_FL = pd.read_parquet('../0_Dataset/Yelp/checkin_reducido.parquet')
df_tip_FL = pd.read_parquet('../0_Dataset/Yelp/tip_reducido.parquet')
df_user_FL = pd.read_parquet('../0_Dataset/Yelp/user_reducido.parquet')

#Google
df_G_review_FL = pd.read_parquet('../0_Dataset/Google/G_review_reducido.parquet')
df_G_metadata_FL = pd.read_parquet('../0_Dataset/Google/G_metadata_FL_reducido.parquet')


___

### Filtramos pizzerias en el estado Florida


Vamos a filtrar por estado. Para eso primero observamos todos los estados y luego elejimos el que vamos a usar

In [ ]:
business['state'].value_counts()

### Filtramos las categorías más frecuentes en general en Estados Unidos


In [ ]:
# Obtener el conteo de cada categoría en el DataFrame completo
conteo_categorias_totales = business['categories'].value_counts()

# Obtener las categorías únicas y ordenarlas alfabéticamente
categorias_unicas_totales = sorted(business['categories'].unique())

# Calcular el top 10 de categorías más frecuentes en general
top_10_categorias_totales = conteo_categorias_totales.head(10)

# Imprimir el top 10 de categorías más frecuentes en general
print("Top 10 Categorías Más Frecuentes en General:")
for i, (categoria, conteo) in enumerate(top_10_categorias_totales.items(), 1):
    print(f"{i}. {categoria}: {conteo} veces")

# Si hay más categorías, mostrar un mensaje de resumen
if len(conteo_categorias_totales) > 10:
    print(f"\nY {len(conteo_categorias_totales) - 10} categorías más...")

print("\n")

### Vamos a observar las categorías relacionadas con restaurantes en Florida


In [ ]:
# Filtrar el DataFrame original para obtener solo las categorías relacionadas con restaurantes en Florida
df_estado_FL = business[business['categories'].str.lower().str.contains('restaurant', na=False) & (business['state'] == 'FL')].reset_index(drop=True)

# Obtener el conteo de cada categoría específicamente en Florida
conteo_categorias_FL = df_estado_FL['categories'].value_counts()

# Calcular el top 10 de categorías más frecuentes en Florida
top_10_categorias_FL = conteo_categorias_FL.head(10)

# Imprimir el top 10 de categorías más frecuentes en Florida
print("Top 10 Categorías Más Frecuentes en Florida:")
for i, (categoria, conteo) in enumerate(top_10_categorias_FL.items(), 1):
    print(f"{i}. {categoria}: {conteo} veces")

# Si hay más categorías, mostrar un mensaje de resumen
if len(conteo_categorias_FL) > 10:
    print(f"\nY {len(conteo_categorias_FL) - 10} categorías más...")

### Filtramos por la el interes del proyecto: Pizzerias

Primero vamos a contar la cantidad de Pizzerias por estado


In [ ]:
# Asegurar que 'categories' es de tipo string
business['categories'] = business['categories'].astype(str)

# Filtra las filas que contienen 'pizza' en 'categories'
pizzerias = business[business['categories'].str.lower().str.contains('pizza', na=False)]

# Cuenta la cantidad de pizzerías en cada estado
conteo_pizzerias_por_estado = pizzerias['state'].value_counts()

# Imprime el resultado
print("Cantidad de Pizzerías por Estado:")
print(conteo_pizzerias_por_estado)


### Filtramos las pizzerias de Florida


In [ ]:
#Filtrar el DataFrame original para obtener solo las categorías relacionadas con restaurantes en Florida
#Solo elegimos los locales abiertos
df_estado_FL = business[business['categories'].str.lower().str.contains('pizza', na=False) & (business['state'] == 'FL') & (business['is_open'] == 1)].reset_index(drop=True)
data_type_check(df_estado_FL)
df_estado_FL.sample(2)

In [ ]:
# Eliminamos state y is_open
df_estado_FL = df_estado_FL.drop(columns=['state', 'is_open'])

In [ ]:
# Filtrar las pizzerías solo para el estado de Florida (FL)
pizzerias_florida = df_estado_FL[df_estado_FL['categories'].str.contains('Pizza', case=False, na=False)]

# Ordenar las pizzerías por stars de manera descendente y luego por review_count de manera descendente
top_pizzerias_por_star_y_reseñas = pizzerias_florida.sort_values(by=['stars', 'review_count'], ascending=[False, False])

# Muestra las primeras 10 pizzerías del top
top_10_pizzerias_por_star_y_reseñas = top_pizzerias_por_star_y_reseñas.head(10)

# Imprime el resultado
print("Top 10 Pizzerías en Florida por Calificación Promedio y Reseñas:")
for i, (index, row) in enumerate(top_10_pizzerias_por_star_y_reseñas.iterrows(), 1):
    print(f"{i}. {row['name']} - {row['city']}, {row['state']} - {row['stars']} estrellas, {row['review_count']} reseñas")


In [ ]:
# Eliminamos state y is_open
df_estado_FL = df_estado_FL.drop(columns=['state', 'is_open'])

In [ ]:
#reseteamos el indice
df_estado_FL = df_estado_FL.reset_index(drop=True)